In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce


In [4]:
# read all the worksheets form the data excel file
df = pd.read_excel('data/Muesli Project raw data.xlsx', header=1)
df1 = pd.read_excel('data/Muesli Project raw data.xlsx',sheet_name=1)
df2 = pd.read_excel('data/Muesli Project raw data.xlsx',sheet_name=2)
df3 = pd.read_excel('data/Muesli Project raw data.xlsx',sheet_name=3)

In [5]:
# check the df headers
print(df.columns)
print(df1.columns)
print(df2.columns)
print(df3.columns)

Index(['Index', 'Order ID', 'Order Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'Origin Channel', 'Country/Region', 'City', 'State',
       'Postal Code', 'Region', 'Category', 'Sub-Category', 'Product ID',
       'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')
Index(['Order ID', 'Arrival Scan Date', 'Customer Name'], dtype='object')
Index(['Row ID', 'Order ID', 'Order Date', 'On Truck Scan Date', 'Ship Mode'], dtype='object')
Index(['Order ID', 'Ready to Ship Date', 'Pickup Date'], dtype='object')


In [6]:
# delete unneeded columns
df = df.drop(['Index','Origin Channel', 'Category', 'Sub-Category'], axis=1)
df1 = df1.drop(['Customer Name'], axis=1)
df2 = df2.drop(['Row ID', 'Order Date'], axis=1)

In [7]:
# check the number of rows and columns
print('df=',df.shape)
print('df1=',df1.shape)
print('df2=',df2.shape)
print('df3=',df3.shape)

df= (9994, 15)
df1= (333, 2)
df2= (5899, 3)
df3= (290, 3)


In [8]:
# check for duplicates
print('df=',df.duplicated().value_counts())
print('-' * 30)
print('df1=',df1.duplicated().value_counts())
print('-' * 30)
print('df2=',df2.duplicated().value_counts())
print('-' * 30)
print('df3=',df3.duplicated().value_counts())

df= False    9993
True        1
Name: count, dtype: int64
------------------------------
df1= False    333
Name: count, dtype: int64
------------------------------
df2= False    3003
True     2896
Name: count, dtype: int64
------------------------------
df3= False    204
True      86
Name: count, dtype: int64


In [9]:
# drop the duplicates
df_no_dups = df.drop_duplicates()
df2_no_dups = df2.drop_duplicates()
df3_no_dups = df3.drop_duplicates()

In [10]:
# check the number of rows and columns
print('df_no_dups=', df_no_dups.shape)
print('df1=', df1.shape)
print('df2_no_dups=', df2_no_dups.shape)
print('df3_no_dups=', df3_no_dups.shape)

df_no_dups= (9993, 15)
df1= (333, 2)
df2_no_dups= (3003, 3)
df3_no_dups= (204, 3)


In [11]:
# check the df headers and decide what to merge with what
print(df_no_dups.columns)
print(df1.columns)
print(df2_no_dups.columns)
print(df3_no_dups.columns)

Index(['Order ID', 'Order Date', 'Ship Mode', 'Customer ID', 'Customer Name',
       'Country/Region', 'City', 'State', 'Postal Code', 'Region',
       'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')
Index(['Order ID', 'Arrival Scan Date'], dtype='object')
Index(['Order ID', 'On Truck Scan Date', 'Ship Mode'], dtype='object')
Index(['Order ID', 'Ready to Ship Date', 'Pickup Date'], dtype='object')


We have to calculate and validate the company assumptions:
1. Preparation process duration from order reception to shipping (2 days): 'Order Date' - 'Ready to Ship Day' (merge df and df3)
2. Duration of process from "Ready to Ship" to "on Truck" (1 day for normal, 0 days for express): merge df2 and df3
3. Efficacy of "Express Processing": the order ought to be on the truck on the ready to ship day
4. From "On Truck Scan" to "Arrival Scan Date"

...and get image of the whole delivery process duration: from 'Order Date' to 'Arrival Scan Date' (merge df and df2)



1. preparation process duration verification

In [12]:
# merge df and df3

df_prep_dur_ver = pd.merge(df_no_dups, df3_no_dups, on='Order ID', how='inner')

In [13]:
# verify the headers of the merged dataframe

df_prep_dur_ver.columns

Index(['Order ID', 'Order Date', 'Ship Mode', 'Customer ID', 'Customer Name',
       'Country/Region', 'City', 'State', 'Postal Code', 'Region',
       'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit',
       'Ready to Ship Date', 'Pickup Date'],
      dtype='object')

In [14]:
# delete unneeded columns

df_prep_dur_ver = df_prep_dur_ver.drop(['Ship Mode', 'Country/Region', 'City', 'State', 
                                        'Postal Code', 'Region', 'Product ID', 
                                        'Sales', 'Quantity', 'Discount', 'Profit'], axis=1)

In [15]:
# verify

df_prep_dur_ver.sample(10)

,Order ID,Order Date,Customer ID,Customer Name,Ready to Ship Date,Pickup Date
296,CA-2019-164350,2019-09-10,CG-12040,Catherine Glotzbach,2019-09-12,2019-09-13
329,CA-2020-147760,2020-11-04,KL-16555,Kelly Lampkin,2020-11-05,2020-11-06
260,CA-2020-163874,2020-11-10,NC-18535,Nick Crebassa,2020-11-12,2020-11-13
248,CA-2020-152737,2020-11-07,TS-21505,Tony Sayre,2020-11-09,2020-11-12
185,US-2020-118157,2020-11-14,AW-10930,Arthur Wiediger,2020-11-19,2020-11-20
364,CA-2020-167941,2020-11-06,JF-15565,Jill Fjeld,2020-11-09,2020-11-12
78,CA-2019-145625,2019-09-11,KC-16540,Kelly Collister,2019-09-17,2019-09-18
365,CA-2020-164168,2020-11-12,LS-16975,Lindsay Shagiari,2020-11-18,2020-11-20
112,CA-2020-110443,2020-11-21,CK-12205,Chloris Kastensmidt,2020-11-27,2020-11-30
84,CA-2020-166436,2020-11-24,TS-21370,Todd Sumrall,2020-11-30,2020-12-02


In [16]:
# check the type of data in the columns

df_prep_dur_ver.dtypes

Order ID                      object
Order Date            datetime64[ns]
Customer ID                   object
Customer Name                 object
Ready to Ship Date    datetime64[ns]
Pickup Date           datetime64[ns]
dtype: object

In [17]:
# calculate the preparation duration and print it in a new column

df_prep_dur_ver['prep duration'] = df_prep_dur_ver['Ready to Ship Date']-df_prep_dur_ver['Order Date']

In [18]:
# verify

df_prep_dur_ver.sample(10)

,Order ID,Order Date,Customer ID,Customer Name,Ready to Ship Date,Pickup Date,prep duration
383,US-2019-147711,2019-09-03,HF-14995,Herbert Flentye,2019-09-09,2019-09-11,6 days
246,CA-2019-100244,2019-09-20,GM-14695,Greg Maxwell,2019-09-24,2019-09-25,4 days
120,CA-2020-104864,2020-11-18,JS-15685,Jim Sink,2020-11-23,2020-11-25,5 days
113,CA-2020-155159,2020-11-25,DL-13315,Delfina Latchford,2020-11-30,2020-12-02,5 days
86,CA-2020-130386,2020-11-12,NG-18430,Nathan Gelder,2020-11-18,2020-11-20,6 days
379,CA-2019-134516,2019-09-20,FM-14215,Filia McAdams,2019-09-25,2019-09-27,5 days
73,US-2019-147711,2019-09-03,HF-14995,Herbert Flentye,2019-09-09,2019-09-11,6 days
282,CA-2020-156622,2020-11-23,JP-15460,Jennifer Patt,2020-11-26,2020-11-27,3 days
288,CA-2020-102925,2020-11-05,CD-12280,Christina DeMoss,2020-11-10,2020-11-12,5 days
257,CA-2020-131254,2020-11-19,NC-18415,Nathan Cano,2020-11-23,2020-11-23,4 days


In [19]:
# check the data type

df_prep_dur_ver['prep duration'].dtype

dtype('<m8[ns]')

what?...

dtype('<m8[ns]') indicates that the data type of the object is a datetime64, representing date and time information with nanosecond precision.

ok, now I get it!

In [20]:
# calculate the mean prep duration
mean_duration_days = df_prep_dur_ver['prep duration'].mean()

# Round the mean duration to the nearest days and hours
rounded_duration = mean_duration_days.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_days = rounded_duration.days
rounded_hours = rounded_duration.seconds // 3600  # Convert seconds to hours

print(f"{rounded_days} days {rounded_hours} hours")

4 days 4 hours


that's way more than the 2 days claimed preparation duration!

2. Duration of process from "Ready to Ship" to "on Truck" (1 day for normal, 0 days for express): merge df2 and df3


In [21]:
# merge df2 and df3

df_dur_readytoship_truck = pd.merge(df2_no_dups, df3_no_dups, on='Order ID', how='inner')

In [22]:
# verify the merged dataframe

df_dur_readytoship_truck.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date
54,CA-2019-134516,2019-09-27,Standard Processing,2019-09-25,2019-09-27
79,CA-2020-127782,2020-11-09,Standard Processing,2020-11-06,2020-11-09
118,US-2020-125808,2020-11-18,Standard Processing,2020-11-16,2020-11-18
185,CA-2020-166436,2020-12-02,Standard Processing,2020-11-30,2020-12-02
110,US-2020-118598,2020-11-16,Express,2020-11-16,2020-11-16
69,CA-2020-147760,2020-11-06,Express,2020-11-05,2020-11-06
150,US-2020-102904,2020-11-23,Standard Processing,2020-11-20,2020-11-23
5,US-2019-101616,2019-09-06,Express,2019-09-06,2019-09-06
91,CA-2020-100622,2020-11-12,Standard Processing,2020-11-09,2020-11-12
149,CA-2020-157420,2020-11-23,Express,2020-11-23,2020-11-23


In [23]:
# calculate the duration of the process from "Ready to Ship" to "on Truck"  and print it in a new column

df_dur_readytoship_truck['ship to truck'] = df_dur_readytoship_truck['On Truck Scan Date']-df_dur_readytoship_truck['Ready to Ship Date']

In [24]:
# verify

df_dur_readytoship_truck.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date,ship to truck
14,CA-2019-102981,2019-09-11,Standard Processing,2019-09-09,2019-09-11,2 days
174,CA-2020-146724,2020-11-30,Standard Processing,2020-11-27,2020-11-30,3 days
44,US-2019-114888,2019-09-25,Standard Processing,2019-09-24,2019-09-25,1 days
132,US-2020-118157,2020-11-20,Express,2020-11-19,2020-11-20,1 days
82,CA-2020-114258,2020-11-12,Standard Processing,2020-11-10,2020-11-12,2 days
25,CA-2019-149223,2019-09-13,Standard Processing,2019-09-11,2019-09-13,2 days
10,CA-2019-126732,2019-09-09,Standard Processing,2019-09-06,2019-09-09,3 days
162,CA-2020-118122,2020-11-25,Standard Processing,2020-11-24,2020-11-25,1 days
58,CA-2019-141551,2019-10-02,Standard Processing,2019-09-30,2019-10-02,2 days
177,CA-2020-105130,2020-11-30,Standard Processing,2020-11-27,2020-11-30,3 days


In [25]:
df_dur_readytoship_truck['ship to truck'].unique()

<TimedeltaArray>
['1 days', '0 days', '3 days', '2 days']
Length: 4, dtype: timedelta64[ns]

In [26]:
# check the type of data in the columns

df_dur_readytoship_truck.dtypes

Order ID                       object
On Truck Scan Date     datetime64[ns]
Ship Mode                      object
Ready to Ship Date     datetime64[ns]
Pickup Date            datetime64[ns]
ship to truck         timedelta64[ns]
dtype: object

In [27]:
# filter for the standard processing

df_dur_readytoship_truck_standard = df_dur_readytoship_truck[df_dur_readytoship_truck['Ship Mode']== 'Standard Processing']

In [28]:
# verify

df_dur_readytoship_truck_standard.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date,ship to truck
74,CA-2020-153339,2020-11-06,Standard Processing,2020-11-05,2020-11-06,1 days
185,CA-2020-166436,2020-12-02,Standard Processing,2020-11-30,2020-12-02,2 days
138,CA-2020-130386,2020-11-20,Standard Processing,2020-11-18,2020-11-20,2 days
60,CA-2019-100944,2019-10-02,Standard Processing,2019-09-30,2019-10-02,2 days
139,CA-2020-164168,2020-11-20,Standard Processing,2020-11-18,2020-11-20,2 days
187,US-2020-128118,2020-12-02,Standard Processing,2020-12-01,2020-12-02,1 days
88,CA-2020-121615,2020-11-12,Standard Processing,2020-11-09,2020-11-12,3 days
87,CA-2020-142636,2020-11-12,Standard Processing,2020-11-09,2020-11-12,3 days
9,CA-2019-106950,2019-09-09,Standard Processing,2019-09-06,2019-09-09,3 days
150,US-2020-102904,2020-11-23,Standard Processing,2020-11-20,2020-11-23,3 days


In [29]:
# calculate the mean prep duration for the standard processing
mean_to_truck_days_standard = df_dur_readytoship_truck_standard['ship to truck'].mean()

# Round the mean duration to the nearest days and hours
rounded_to_truck_dur_standard = mean_to_truck_days_standard.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_to_truck_days_standard = rounded_to_truck_dur_standard.days
rounded_to_truck_hours_standard = rounded_to_truck_dur_standard.seconds // 3600  # Convert seconds to hours

print(f"{rounded_to_truck_days_standard} days {rounded_to_truck_hours_standard} hours")

2 days 0 hours


...that's more than the claimed one day

3. Efficacy of "Express Processing": the order ought to be on the truck on the ready to ship day


In [30]:
# filter for the express processing

df_dur_readytoship_truck_xpress = df_dur_readytoship_truck[df_dur_readytoship_truck['Ship Mode']== 'Express']

In [31]:
# verify

df_dur_readytoship_truck_xpress.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date,ship to truck
0,CA-2019-116540,2019-09-03,Express,2019-09-02,2019-09-03,1 days
98,US-2020-116491,2020-11-13,Express,2020-11-13,2020-11-13,0 days
5,US-2019-101616,2019-09-06,Express,2019-09-06,2019-09-06,0 days
99,US-2020-116659,2020-11-13,Express,2020-11-12,2020-11-13,1 days
144,CA-2020-162929,2020-11-23,Express,2020-11-23,2020-11-23,0 days
105,CA-2020-115448,2020-11-16,Express,2020-11-16,2020-11-16,0 days
2,CA-2019-129630,2019-09-04,Express,2019-09-04,2019-09-04,0 days
143,US-2020-132444,2020-11-23,Express,2020-11-23,2020-11-23,0 days
69,CA-2020-147760,2020-11-06,Express,2020-11-05,2020-11-06,1 days
64,US-2020-168116,2020-11-04,Express,2020-11-04,2020-11-04,0 days


In [32]:
# calculate the mean prep duration for the express processing
mean_to_truck_days_xpress = df_dur_readytoship_truck_xpress['ship to truck'].mean()

# Round the mean duration to the nearest days and hours
rounded_to_truck_dur_xpress = mean_to_truck_days_xpress.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_to_truck_days_xpress = rounded_to_truck_dur_xpress.days
rounded_to_truck_hours_xpress = rounded_to_truck_dur_xpress.seconds // 3600  # Convert seconds to hours

print(f"{rounded_to_truck_days_xpress} days {rounded_to_truck_hours_xpress} hours")

0 days 10 hours


no comments

4a. From "On Truck Scan" to "Arrival Scan Date"


In [33]:
# merge df1 and df2

df_order_del_time = pd.merge(df2_no_dups, df1, on='Order ID', how='inner')

In [34]:
# verify

df_order_del_time.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Arrival Scan Date
148,CA-2019-108644,2019-10-04,Express,2019-10-07
262,CA-2019-113600,2019-12-20,Standard Processing,2019-12-23
299,CA-2020-120019,2020-02-26,Standard Processing,2020-03-02
285,CA-2020-149076,2020-01-22,Standard Processing,2020-01-27
218,CA-2019-124485,2019-11-27,Express,2019-12-02
239,CA-2019-105018,2019-12-04,Standard Processing,2019-12-09
279,CA-2019-122017,2020-01-06,Standard Processing,2020-01-10
154,US-2019-162026,2019-10-09,Express,2019-10-14
35,CA-2019-164672,2019-05-15,Standard Processing,2019-05-20
266,CA-2019-168536,2019-12-24,Standard Processing,2019-12-30


In [35]:
df_order_del_time['delivery time']=df_order_del_time['Arrival Scan Date']- df_order_del_time['On Truck Scan Date']

In [36]:
# verify
df_order_del_time.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Arrival Scan Date,delivery time
47,CA-2019-141180,2019-05-22,Standard Processing,2019-05-27,5 days
175,CA-2019-125738,2019-10-23,Standard Processing,2019-10-28,5 days
6,CA-2019-109057,2019-05-01,Standard Processing,2019-05-07,6 days
112,CA-2019-121223,2019-09-16,Standard Processing,2019-09-19,3 days
108,US-2019-152373,2019-09-13,Standard Processing,2019-09-17,4 days
237,US-2019-163881,2019-12-04,Standard Processing,2019-12-06,2 days
328,CA-2020-125381,2020-05-01,Standard Processing,2020-05-08,7 days
294,CA-2020-131828,2020-02-14,Standard Processing,2020-02-18,4 days
69,CA-2019-143714,2019-05-29,Standard Processing,2019-06-03,5 days
168,CA-2019-113390,2019-10-16,Standard Processing,2019-10-21,5 days


In [37]:
# calculate the mean prep duration for the standard processing
mean_order_del_time = df_order_del_time['delivery time'].mean()

# Round the mean duration to the nearest days and hours
rounded_mean_del_time = mean_order_del_time.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_mean_del_days = rounded_mean_del_time.days
rounded_mean_del_hours = rounded_mean_del_time.seconds // 3600  # Convert seconds to hours

print(f"{rounded_mean_del_days} days {rounded_mean_del_hours} hours")

4 days 14 hours


the shipping company claim is three days, oups!

4b. From "Pickup Date" to "Arrival Scan Date"

In [38]:
# merge df1 and df3

df_order_del_time2 = pd.merge(df1, df3_no_dups, on='Order ID', how='inner')

In [39]:
# verify

df_order_del_time2.sample(10)

,Order ID,Arrival Scan Date,Ready to Ship Date,Pickup Date
6,CA-2019-102981,2019-09-16,2019-09-09,2019-09-11
20,CA-2019-141551,2019-10-07,2019-09-30,2019-10-02
19,CA-2019-166772,2019-09-30,2019-09-24,2019-09-25
18,CA-2019-145611,2019-09-30,2019-09-24,2019-09-25
8,CA-2019-133123,2019-09-16,2019-09-11,2019-09-13
3,CA-2019-158099,2019-09-11,2019-09-05,2019-09-06
9,CA-2019-149223,2019-09-17,2019-09-11,2019-09-13
0,CA-2019-129630,2019-09-09,2019-09-04,2019-09-04
12,CA-2019-146206,2019-09-23,2019-09-16,2019-09-18
21,CA-2019-163202,2019-10-07,2019-10-02,2019-10-04


In [40]:
df_order_del_time2['delivery time2']=df_order_del_time2['Arrival Scan Date']- df_order_del_time2['Pickup Date']

In [41]:
# verify

df_order_del_time2.sample(10)

,Order ID,Arrival Scan Date,Ready to Ship Date,Pickup Date,delivery time2
14,CA-2019-130225,2019-09-23,2019-09-17,2019-09-18,5 days
15,CA-2019-102932,2019-09-30,2019-09-23,2019-09-25,5 days
17,CA-2019-100244,2019-09-30,2019-09-24,2019-09-25,5 days
1,CA-2019-129847,2019-09-10,2019-09-04,2019-09-04,6 days
9,CA-2019-149223,2019-09-17,2019-09-11,2019-09-13,4 days
10,CA-2019-121223,2019-09-19,2019-09-13,2019-09-16,3 days
12,CA-2019-146206,2019-09-23,2019-09-16,2019-09-18,5 days
3,CA-2019-158099,2019-09-11,2019-09-05,2019-09-06,5 days
20,CA-2019-141551,2019-10-07,2019-09-30,2019-10-02,5 days
11,US-2019-118780,2019-09-23,2019-09-16,2019-09-18,5 days


In [42]:
# calculate the mean prep duration for the standard processing
mean_order_del_time2 = df_order_del_time2['delivery time2'].mean()

# Round the mean duration to the nearest days and hours
rounded_mean_del_time2 = mean_order_del_time2.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_mean_del2_days = rounded_mean_del_time2.days
rounded_mean_del2_hours = rounded_mean_del_time2.seconds // 3600  # Convert seconds to hours

print(f"{rounded_mean_del2_days} days {rounded_mean_del2_hours} hours")

4 days 16 hours



...and get image of the whole delivery process duration: from 'Order Date' to 'Arrival Scan Date' (merge df and df2)

In [43]:
# merge df and df1

df_order_del_total = pd.merge(df1, df_no_dups, on='Order ID', how='inner')

In [44]:
# verify

df_order_del_total.sample(10)

,Order ID,Arrival Scan Date,Order Date,Ship Mode,Customer ID,Customer Name,Country/Region,City,State,Postal Code,Region,Product ID,Sales,Quantity,Discount,Profit
11,CA-2019-162138,2019-05-06,2019-04-23,Standard Class,GK-14620,Grace Kelly,United States,Hesperia,California,92345.0,West,OFF-BI-10004593,251.520,6,0.2,81.7440
24,CA-2019-109869,2019-05-07,2019-04-22,Standard Class,TN-21040,Tanja Norvell,United States,Phoenix,Arizona,85023.0,West,FUR-FU-10000023,23.560,5,0.2,7.0680
368,CA-2019-143441,2019-11-11,2019-11-05,First Class,EB-14170,Evan Bailliet,United States,Laredo,Texas,78041.0,Central,OFF-LA-10002312,11.840,1,0.2,4.4400
133,CA-2019-133725,2019-06-03,2019-05-23,Standard Class,KL-16645,Ken Lonsdale,United States,Chicago,Illinois,60623.0,Central,TEC-PH-10004165,1979.928,9,0.2,148.4946
130,CA-2019-114209,2019-06-03,2019-05-21,Standard Class,AS-10285,Alejandro Savely,United States,Dallas,Texas,75081.0,Central,OFF-BI-10000343,1.964,2,0.8,-3.2406
378,CA-2019-161669,2019-11-18,2019-11-07,First Class,EM-14095,Eudokia Martin,United States,Los Angeles,California,90036.0,West,OFF-LA-10004093,14.620,2,0.0,6.8714
282,CA-2019-149979,2019-10-07,2019-09-23,Second Class,RA-19915,Russell Applegate,United States,Columbus,Georgia,31907.0,South,FUR-CH-10002084,368.970,3,0.0,81.1734
605,CA-2020-104885,2020-03-16,2020-03-02,Standard Class,DB-13555,Dorothy Badders,United States,Newark,Delaware,19711.0,East,FUR-BO-10003660,441.960,2,0.0,101.6508
7,CA-2019-138933,2019-05-03,2019-04-24,First Class,JL-15130,Jack Lebron,United States,Riverside,California,92503.0,West,OFF-BI-10003355,3.984,1,0.2,1.3944
530,CA-2019-148684,2020-01-06,2019-12-25,Standard Class,TS-21655,Trudy Schmidt,United States,Fayetteville,Arkansas,72701.0,South,FUR-FU-10003553,343.850,5,0.0,137.5400


In [45]:
df_order_del_total = df_order_del_total.drop(['Ship Mode', 'Country/Region', 'City', 'State', 
                                        'Postal Code', 'Region', 'Product ID', 
                                        'Sales', 'Quantity', 'Discount', 'Profit'], axis=1)

In [46]:
# verify

df_order_del_total.sample(10)

,Order ID,Arrival Scan Date,Order Date,Customer ID,Customer Name
468,CA-2019-100083,2019-12-09,2019-11-24,CD-11980,Carol Darley
602,CA-2020-104885,2020-03-16,2020-03-02,DB-13555,Dorothy Badders
417,CA-2019-134110,2019-11-25,2019-11-17,BG-11035,Barry Gonzalez
424,CA-2019-164511,2019-11-29,2019-11-19,DJ-13630,Doug Jacobs
629,CA-2020-124716,2020-04-06,2020-03-27,BD-11560,Brendan Dodson
483,CA-2019-143749,2019-12-13,2019-12-05,AG-10300,Aleksandra Gannaway
587,CA-2020-104801,2020-02-24,2020-02-13,FH-14350,Fred Harton
371,CA-2019-157280,2019-11-11,2019-11-05,LW-17125,Liz Willingham
356,CA-2019-125815,2019-11-08,2019-10-28,DL-13330,Denise Leinenbach
446,CA-2019-128531,2019-12-03,2019-11-24,NS-18505,Neola Schneider


In [47]:
df_order_del_total['total delivery time']=df_order_del_total['Arrival Scan Date']- df_order_del_total['Order Date']

In [48]:
# verify

df_order_del_total.sample(10)

,Order ID,Arrival Scan Date,Order Date,Customer ID,Customer Name,total delivery time
129,CA-2019-114209,2019-06-03,2019-05-21,AS-10285,Alejandro Savely,13 days
262,CA-2019-100244,2019-09-30,2019-09-20,GM-14695,Greg Maxwell,10 days
300,US-2019-140158,2019-10-11,2019-10-04,DR-12940,Daniel Raglin,7 days
579,CA-2020-117870,2020-02-06,2020-01-27,JH-15820,John Huston,10 days
293,CA-2019-163202,2019-10-07,2019-09-27,BM-11650,Brian Moss,10 days
424,CA-2019-164511,2019-11-29,2019-11-19,DJ-13630,Doug Jacobs,10 days
248,CA-2019-105900,2019-09-30,2019-09-17,BS-11590,Brendan Sweed,13 days
194,CA-2019-149195,2019-09-13,2019-09-05,DM-13525,Don Miller,8 days
359,CA-2019-156811,2019-11-11,2019-10-27,BP-11095,Bart Pistole,15 days
203,CA-2019-149783,2019-09-16,2019-09-04,DL-13315,Delfina Latchford,12 days


In [49]:
# calculate the mean prep duration for the standard processing
mean_order_del_total = df_order_del_total['total delivery time'].mean()

# Round the mean duration to the nearest days and hours
rounded_mean_del_total = mean_order_del_total.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_mean_del_total_days = rounded_mean_del_total.days
rounded_mean_del_total_hours = rounded_mean_del_total.seconds // 3600  # Convert seconds to hours

print(f"{rounded_mean_del_total_days} days {rounded_mean_del_total_hours} hours")

10 days 21 hours


In [50]:
# merge the dataframes with the 'reduce' function
dfs = [df, df1, df2_red_no_dups, df3_no_dups]

merged_df_red = reduce(lambda left, right: pd.merge(left, right, on='Order ID', how='inner'), dfs)

merged_df_red.shape

NameError: name 'df2_red_no_dups' is not defined

In [51]:
%store df_order_del_time2

Stored 'df_order_del_time2' (DataFrame)
